In [1]:
#setting libraries
import os
import re
import datetime

In [19]:
#reading data and separating data into test and bench files
tests_files=[a for a in os.listdir('./data.nogit/') if 'test' in a]
bench_files=[a for a in os.listdir('./data.nogit/') if 'bench' in a]
# os.getcwd()

In [ ]:
#overview of the test files
bits={} #key:file name value:number of bits

for a in tests_files:
    total_bits=0
    n_input=0
    n_outpus=0
    with open(f'./data.nogit/{a}','r') as file:
        while True:
            line=file.readline().strip()
            if not(line):
                break
            else: #there's available data to read
                total_bits=max(total_bits,len(line))
                n_outpus=line.count('-')
                n_input=len(line)-n_outpus
    bits[a]={"total_bits":total_bits,"INPUT":n_input,"OUTPUT":n_outpus}
    file.close()

for a,b in bits.items():
    print(f'file: {a}, total bits to work: {b}')

In [ ]:
def extract_number_from_string(string:str):
    '''
    :param str string: the string of values to evaluate
    input string to extract the integer value
    '''
    print(string)
    aux=re.findall(r'(\d+) \w+',string)
    print(aux)
    return int(aux[0])
#overview of bench files

n_vales={} #key:file_name value:{key: input/output/gates value:read value from file}
exp_values={} #key: input/output/gates value:expected number
for a in bench_files:
    n_vales[a]={}
    exp_values[a]={}
    aux=0
    print(f'read file: {a}')
    with open(f'./data.nogit/{a}','r') as file:
        while True:
            line=file.readline().strip()
            if aux>3:
                break
            if not(line):
                aux+=1           
                continue
            else:
                if line[0]=='#': #it show the information about the file
                    try:
                        exp_values[a][line.split(' ')[2].upper()]=extract_number_from_string(line.lower())
                    except:
                        continue
                else:
                    gate_str="".join(filter(str.isalpha,line))
                    if gate_str not in n_vales[a]:
                        n_vales[a][gate_str]=0
                    n_vales[a][gate_str]+=1
    file.close()


In [ ]:
gates=[] #getting the names of the gates used in the bench files
for a,b in n_vales.items():
    print (a,b)
    for c  in b.keys():
        if c not in ['INPUT','OUTPUT']:
            if c not in gates:
                gates.append(c)

print(gates)

Code for running tests
Testing will be done with file c17.tests and c17.bench

In [2]:
#setting time to Japan time (UTC+9)
jpTime=datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
#when testing started
t0=jpTime.now()

In [3]:
def log(msg:str):
    '''
    :param str msg: the message to appear in the log
    '''
    delta=jpTime.now()-t0 #show the running time
    print(f'{delta}...{msg}')

In [5]:
#logic gate tryout
def logic_gate(gate:str, inputs:list):
    '''
    defining the logic gate functioning

    Params:
        gate(str): 'NAND', 'AND', 'OR', 'NOT', 'NOR', 'BUFF', 'XOR' name of gates in bench files
        inputs(list): defines the inputs for the gate [A,B,C...], the number of elements in the function will determine the amount of pins for the logic gate.EXCEPT for NOT gates it will take ONLY the first element

    Returns:
        (bool): a single logic value to assign to the corresponding 
    '''
    #verifying that the input list is just integers 0/1
    aux,inputs=inputs,[]
    inputs=[int(x) for x in aux]
    if gate=='NAND':
        holder=1
        [holder:=holder&x for x in inputs]
        return 0 if holder else 1
    elif gate=="AND":
        holder=1
        [holder:=holder&x for x in inputs]
        return 1 if holder else 0
    elif gate=="OR":
        holder =0
        [holder:=holder|x for x in inputs]
        return 1 if holder else 0
    elif gate=="NOT":
        return 0 if inputs[0] else 1
    elif gate=="NOR":
        holder =0
        [holder:=holder|x for x in inputs]
        return 0 if holder else 1
    elif gate=="BUFF":
        return 1 if inputs[0] else 0
    elif gate=="XOR":
        holder=0
        [holder:=holder^x for x in inputs]
        return 0 if holder else 1

In [5]:
#reading the bench file to assign the input, output and intermediate connections.
#read only once (the rutine is the same for ALL the data)
def read_bench(bench_file:str):
    aux=0
    signal_line={} #key:singal line value: logic value (0/1)
    primary_input=[]
    primary_output=[]
    circ_struc=[] #circuit structure
    with open(bench_file) as f:
        while True:
            a=f.readline().strip()
            if aux>3:
                break
            if not(a):
                aux+=1
                continue
            else:
                if "#" in a:
                    continue
                else:
                    if 'INPUT' in a or 'OUTPUT' in a: #connections for primary input and primary output
                        b=''.join(filter(str.isnumeric,a))
                        signal_line[b]=''
                        if 'INPUT' in a:
                            primary_input.append(b)
                        else:
                            primary_output.append(b)
                    else: # signal = gate (input signal)
                        signal_line[a.split(' = ')[0]]=''
                        circ_struc.append(a)
    f.close()
    log (f"Finished reading bench file: {bench_file}")
    return signal_line, circ_struc, primary_input, primary_output

In [6]:
#circuit code
def circuit_code(signal_line:dict, circ_struc:list):
    # log(f"beginning to run circuit signal input: {signal_line}")
    for a in circ_struc:
        output_signal=a.split(' = ')[0]
        gate=a.split(' = ')[1].split('(')[0]
        input_signal_index=a.split(' = ')[1].split('(')[1][:-1]
        input_signals=[signal_line[x.strip()] for x in input_signal_index.split(',')]
        signal_line[output_signal]=logic_gate(gate,input_signals)
    return signal_line

In [13]:
#reading the test file for data and testing
#this script produces the "correct" behaviour
def read_test(test_file,signal_line,circ_struc,primary_input,primary_output):
    with open(test_file) as f:
        while True:
            line=f.readline().strip()
            if not(line):
                break
            else: #I assume the data is organized in the bench stated order
                #assigning the test data to the corresponding signals
                for a in range(len(primary_input)):
                    try:
                        signal_line[primary_input[a]]=int(line[a])
                    except:
                        continue
                
                #executing the testing from the bench file (running the circuit)
                #structure= output signal= logic gate (input signal)
                signal_line=circuit_code(signal_line,circ_struc)
                
                circuit_output=[signal_line[x] for x in primary_output]
                print(f'Data input: {line[:line.index('-')]}, \tData output: {circuit_output}')
    f.close()
    log(f"Finished running test file {test_file}")
    return circuit_output

In [16]:
# for a in bench_files:
#     print(a)
b,c,d,e=read_bench(f'./data.nogit/c6288.bench')
print(read_test(f'./data.nogit/c6288.tests',b,c,d,e))

0:09:10.336867...Finished reading bench file: ./data.nogit/c6288.bench
Data input: 01010101011101111011000101001010, 	Data output: [0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1]
Data input: 01010100001001111100001011111111, 	Data output: [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]
Data input: 10110100110101100001101110001100, 	Data output: [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]
Data input: 10110000011100000000110111000110, 	Data output: [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
Data input: 11111100100000011010001010111000, 	Data output: [1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0]
Data input: 11111111100011100000000010000001, 	Data output: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,

Code to draw the circuit in the Wokwi

In [ ]:
#logic gates codes to draw circuits in wokwi
def logic_gate_diagram(gate:str,number:int,top:int,left:int):
    '''
    Params:
        gate(str): the logic gate desired
        number(int): the number for id
        top(int): top value for wokwi reference system
        left(int): left value for wokwi reference system
    Returns:
        part_json(str): the corresponding json to draw the circuit on Wokwi
        output_json(str): the correspondin json to indicate the ouput\
    '''
    #NAND gate
    if gate=='NAND':
        type="wokwi-gate-nand-2"
    if gate=='AND':
        type="wokwi-gate-and-2"
    if gate=='NOR':
        type= "wokwi-gate-nor-2"
    if gate=='OR':
        type="wokwi-gate-or-2"
    if gate=='X0R':
        type="wokwi-gate-xor-2"
    if gate=='NOT':
        type="wokwi-gate-not"
    if gate=='BUFF':
        type="wokwi-gate-buffer"
    return f'"type":"{type}", "id":"{gate.lower()}{str(number)}","top":{top},"left":{left}'

In [82]:
#logic gates connections
#[start, finish, color, ["h","v"]]
[ "sw1:1b", "nand1:A", "black", [ "h9.3", "v-9.6" ] ],

#signal number
{ "type": "wokwi-text", "id": "text16", "top": -384, "left": 0, "attrs": { "text": "16" } },

#spacing between logic gates
#same left level gates
#gate and output signal text delta_top=0 delta_left=100



({'type': 'wokwi-text',
  'id': 'text16',
  'top': -384,
  'left': 0,
  'attrs': {'text': '16'}},)

In [ ]:
signal_scale={} #key: level #value[list]: list of inputs. All signals are considered input
signal_scale[0]=d #primary inputs are level 0
for a in c:
    ind=0 #latest index
    output_signal=a.split(' = ')[0]
    #I'm asumming the signal are ordered in ascendent order from the primary inputs.
    # this works for both numbers and letters (for letters it goes alphabetically order a<b<c<d....)
    y=0
    for z in a.split(' = ')[1].split('(')[1][:-1].split(', '):
        y=max(y,int(z))
    z=str(y)

    #evaluates in string format
    while True:
        if z not in signal_scale[ind]:
            ind+=1
        else:
            break
    ind+=1
    if ind not in signal_scale:
        signal_scale[ind]=[]
    signal_scale[ind].append(output_signal)


In [ ]:
# for a,b in enumerate(c):
#     #determine logic gate
#     # gate=b.split(' = ')[1].split('(')[0]
    
#     #extract the inputs
#     #determines if both inputs from the gate are primary inputs
#     # gate_input=b.split(' = ')[1].split('(')[1][:-1].split(', ')
#     # print("{",logic_gate_diagram(gate,a,top,left),"},")
#     #IF both inputs depend on primary input, set them in column 0, if only 1 depends on primary input set column=1, if none, set column=2
    


In [34]:
wokwi_diagram_json={
    "version":1,
    "author":"",
    "editor":"wokwi",
    "parts":[], #logic gates and text
    "connnections":[],#wires between logic gates
    "dependencies":{}
}

Stuck-at fault code

In [8]:
def truth_table(primary_inputs:list, primary_outputs:list,circ_struc:list):
    '''
    Params:
        primary_inputs(list): A list of the primary input signals index
        primary_outputs(list): A list of the primary output signals index
        circ_struct(list): A list with the relationships between inputs and outputs
    Returns:
        fault_free_circuit(dict): key:binary input for primary inputs value:fault-free output
    '''
    log("Beginning truth table generation")
    b={}
    fault_free_circuit={}
    for a in range(2**len(primary_inputs)):
        aux=0
        while aux<len(primary_inputs):
            try:
                b[primary_inputs[aux]]=int(bin(a)[aux+2])
            except:
                b[primary_inputs[aux]]=0
            aux+=1
        normal_output=circuit_code(b,circ_struc)
        bench_outputs=[normal_output[x] for x in primary_outputs]
        print(a,bench_outputs)
        fault_free_circuit['0'*(len(primary_inputs)-(len(bin(a))-2))+bin(a)[2:]]=bench_outputs
    log("Finished generating truth table")
    return fault_free_circuit

In [13]:
a='c17.bench'
signal,circuit,pi,po=read_bench(f'./data.nogit/{a}')
fault_free=truth_table(pi,po,circuit)

0:00:08.066247...Finished reading bench file: ./data.nogit/c17.bench
0:00:08.066405...Beginning truth table generation
0 [0, 0]
1 [0, 0]
2 [0, 0]
3 [1, 1]
4 [0, 0]
5 [1, 0]
6 [1, 1]
7 [1, 1]
8 [0, 0]
9 [0, 0]
10 [1, 0]
11 [1, 0]
12 [1, 1]
13 [1, 1]
14 [1, 1]
15 [1, 0]
16 [0, 0]
17 [0, 1]
18 [0, 0]
19 [0, 1]
20 [1, 0]
21 [1, 1]
22 [1, 0]
23 [1, 0]
24 [1, 1]
25 [1, 1]
26 [1, 1]
27 [1, 1]
28 [1, 1]
29 [1, 1]
30 [1, 0]
31 [1, 0]
0:00:08.068788...Finished generating truth table


Fault equivalency simulations for every logic gate

For any gate try SA0 and SA1 at both input and output

In [116]:
def brute_SA_faults(gate:str,inputs:str|int):
    '''
    Params:
        gate(str): the logic gate to evaluate
        inputs(str|int): is the number of inputs for the gate str:'0000' int:2
    Returns:
    '''

    # inputs=2
    # gate='NAND'
    if isinstance(inputs,int):
        lenght=inputs
    elif isinstance(inputs,str):
        lenght=len(inputs)
    else:
        # return "the provided input is not in a valid format"
        print("the provided input is not in a valid format")

    test='SA0'
    failure=''
    signal=['A','B','Output']
    #SA for the inputs
    for a in range(2**lenght+2):
        for b in range(2**lenght):
            pi_input=bin(b)[2:].zfill(lenght)
            po=logic_gate(gate,list(pi_input))
            sa_input=pi_input[0:(a//2)]+str(int(test[-1]))+pi_input[(a//2)+1:]
            if a<2**lenght:
                sa_output=logic_gate(gate,list(sa_input))
            else:
                sa_input='xx'
                sa_output=int(test[-1])
            
            if sa_output!=po:
                failure="failed"
            else:
                failure=''
            print(f'|{gate}|{pi_input}|{test}[{signal[a//2]}]|{sa_input}|{po}|{sa_output}|{failure}|')
        if test=='SA0':
            test='SA1'
        else:
            test='SA0'

In [118]:
#making a 2 bit truth table 
aux=0
l_gates=['NAND', 'AND', 'OR', 'NOT', 'NOR', 'BUFF', 'XOR']
while aux<len(l_gates):
    print('|Logic Gate|primary input|SA type|stuck-at fault input|primary output|stuck-at output|failed?|')
    print('| :---: '*7)
    print(brute_SA_faults(l_gates[aux],2))
    print('-'*30)
    aux+=1

|Logic Gate|primary input|SA type|stuck-at fault input|primary output|stuck-at output|failed?|
| :---: | :---: | :---: | :---: | :---: | :---: | :---: 
|NAND|00|SA0[A]|00|1|1||
|NAND|01|SA0[A]|01|1|1||
|NAND|10|SA0[A]|00|1|1||
|NAND|11|SA0[A]|01|0|1|failed|
|NAND|00|SA1[A]|10|1|1||
|NAND|01|SA1[A]|11|1|0|failed|
|NAND|10|SA1[A]|10|1|1||
|NAND|11|SA1[A]|11|0|0||
|NAND|00|SA0[B]|00|1|1||
|NAND|01|SA0[B]|00|1|1||
|NAND|10|SA0[B]|10|1|1||
|NAND|11|SA0[B]|10|0|1|failed|
|NAND|00|SA1[B]|01|1|1||
|NAND|01|SA1[B]|01|1|1||
|NAND|10|SA1[B]|11|1|0|failed|
|NAND|11|SA1[B]|11|0|0||
|NAND|00|SA0[Output]|xx|1|0|failed|
|NAND|01|SA0[Output]|xx|1|0|failed|
|NAND|10|SA0[Output]|xx|1|0|failed|
|NAND|11|SA0[Output]|xx|0|0||
|NAND|00|SA1[Output]|xx|1|1||
|NAND|01|SA1[Output]|xx|1|1||
|NAND|10|SA1[Output]|xx|1|1||
|NAND|11|SA1[Output]|xx|0|1|failed|
None
------------------------------
|Logic Gate|primary input|SA type|stuck-at fault input|primary output|stuck-at output|failed?|
| :---: | :---: | :---: | :--